In [1]:
import pandas as pd
import pm4py
from logview.utils import LogViewBuilder

# Load data
bpi_data = pd.read_csv("C:/Users/cshek/OneDrive/Bureaublad/Thesis/BPI_Challenge_2017.csv", sep=',', quotechar='"')
bpi_data.columns = bpi_data.columns.str.strip()
bpi_data['time'] = pd.to_datetime(bpi_data['time'], format='%Y/%m/%d %H:%M:%S.%f')
log = pm4py.format_dataframe(bpi_data, case_id='case', activity_key='event', timestamp_key='time')
display(log)

# Build LogView
log_view = LogViewBuilder.build_log_view(log)

,case,event,time,lifecycle:transition,ApplicationType,LoanGoal,RequestedAmount,MonthlyCost,org:resource,Selected,...,Accepted,CreditScore,NumberOfTerms,EventOrigin,OfferedAmount,case:concept:name,concept:name,time:timestamp,@@index,@@case_index
0,Application_1000086665,A_Create Application,2016-08-03 17:57:21.673000+00:00,COMPLETE,New credit,"Other, see explanation",5000.0,NaN,User_1,NaN,...,NaN,NaN,NaN,Application,NaN,Application_1000086665,A_Create Application,2016-08-03 17:57:21.673000+00:00,0,0
1,Application_1000086665,A_Submitted,2016-08-03 17:57:21.734000+00:00,COMPLETE,New credit,"Other, see explanation",5000.0,NaN,User_1,NaN,...,NaN,NaN,NaN,Application,NaN,Application_1000086665,A_Submitted,2016-08-03 17:57:21.734000+00:00,1,0
2,Application_1000086665,W_Handle leads,2016-08-03 17:57:21.963000+00:00,SCHEDULE,New credit,"Other, see explanation",5000.0,NaN,User_1,NaN,...,NaN,NaN,NaN,Workflow,NaN,Application_1000086665,W_Handle leads,2016-08-03 17:57:21.963000+00:00,2,0
3,Application_1000086665,W_Handle leads,2016-08-03 17:58:28.286000+00:00,WITHDRAW,New credit,"Other, see explanation",5000.0,NaN,User_1,NaN,...,NaN,NaN,NaN,Workflow,NaN,Application_1000086665,W_Handle leads,2016-08-03 17:58:28.286000+00:00,3,0
4,Application_1000086665,W_Complete application,2016-08-03 17:58:28.293000+00:00,SCHEDULE,New credit,"Other, see explanation",5000.0,NaN,User_1,NaN,...,NaN,NaN,NaN,Workflow,NaN,Application_1000086665,W_Complete application,2016-08-03 17:58:28.293000+00:00,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1202262,Application_999993812,W_Call incomplete files,2016-10-20 10:19:28.812000+00:00,RESUME,New credit,Caravan / Camper,30000.0,NaN,User_41,NaN,...,NaN,NaN,NaN,Workflow,NaN,Application_999993812,W_Call incomplete files,2016-10-20 10:19:28.812000+00:00,1202262,31508
1202263,Application_999993812,W_Call incomplete files,2016-10-20 10:21:59.667000+00:00,SUSPEND,New credit,Caravan / Camper,30000.0,NaN,User_41,NaN,...,NaN,NaN,NaN,Workflow,NaN,Application_999993812,W_Call incomplete files,2016-10-20 10:21:59.667000+00:00,1202263,31508
1202264,Application_999993812,O_Accepted,2016-10-24 08:24:30.056000+00:00,COMPLETE,New credit,Caravan / Camper,30000.0,NaN,User_68,NaN,...,NaN,NaN,NaN,Offer,NaN,Application_999993812,O_Accepted,2016-10-24 08:24:30.056000+00:00,1202264,31508
1202265,Application_999993812,A_Pending,2016-10-24 08:24:30.059000+00:00,COMPLETE,New credit,Caravan / Camper,30000.0,NaN,User_68,NaN,...,NaN,NaN,NaN,Application,NaN,Application_999993812,A_Pending,2016-10-24 08:24:30.059000+00:00,1202265,31508


In [2]:
from logview.predicate import Query, EqToConstant, NotEqToConstant, GreaterEqualToConstant, LessThanConstant, StartWith, EndWith, DurationWithin

# CreditScore ≥ 600
query_1 = Query('GoodCredit', [GreaterEqualToConstant('CreditScore', 600)])
result_set_1, complement_1 = log_view.evaluate_query('rs_GoodCredit', log, query_1)
log_view.label_result_set(result_set_1, 'GoodCredit')

# RequestedAmount ≥ 10000
query_2 = Query('LoanOverThreshold', [GreaterEqualToConstant('RequestedAmount', 10000)])
result_set_2, complement_2 = log_view.evaluate_query('rs_LoanOverThreshold', result_set_1, query_2)
log_view.label_result_set(result_set_2, 'LoanOverThreshold')

# RequestedAmount < 15000
query_3 = Query('SmallAmount', [LessThanConstant('RequestedAmount', 15000)])
result_set_3, complement_3 = log_view.evaluate_query('rs_SmallAmount', result_set_2, query_3)
log_view.label_result_set(result_set_3, 'SmallAmount')

# ApplicationType = 'New credit'
query_4 = Query('IsNewCredit', [EqToConstant('ApplicationType', 'New credit')])
result_set_4, complement_4 = log_view.evaluate_query('rs_IsNewCredit', result_set_3, query_4)
log_view.label_result_set(result_set_4, 'IsNewCredit')

# Starts with A_Create Application
query_5 = Query('StartWithCreate', [StartWith(['A_Create Application'])])
result_set_5, complement_5 = log_view.evaluate_query('rs_StartWithCreate', log, query_5)
log_view.label_result_set(result_set_5, 'StartWithCreate')

# Duration between 2 and 7 days
query_6 = Query('ModerateDuration', [DurationWithin(172800, 604800)])
result_set_6, complement_6 = log_view.evaluate_query('rs_ModerateDuration', log, query_6)
log_view.label_result_set(result_set_6, 'ModerateDuration')

# Summary
summary = log_view.get_summary()


+----+----------------------+-------------------+----------------------+-----------------------+
|    | source_log           | query             | result_set           | labels                |
|----+----------------------+-------------------+----------------------+-----------------------|
|  0 | initial_source_log   | GoodCredit        | rs_GoodCredit        | ['GoodCredit']        |
|  1 | rs_GoodCredit        | LoanOverThreshold | rs_LoanOverThreshold | ['LoanOverThreshold'] |
|  2 | rs_LoanOverThreshold | SmallAmount       | rs_SmallAmount       | ['SmallAmount']       |
|  3 | rs_SmallAmount       | IsNewCredit       | rs_IsNewCredit       | ['IsNewCredit']       |
|  4 | initial_source_log   | StartWithCreate   | rs_StartWithCreate   | ['StartWithCreate']   |
|  5 | initial_source_log   | ModerateDuration  | rs_ModerateDuration  | ['ModerateDuration']  |
+----+----------------------+-------------------+----------------------+-----------------------+
+----+-------------------+----

In [3]:
registry = log_view.query_registry.summary()
display(registry)


{'evaluations':              source_log              query            result_set  \
 0    initial_source_log         GoodCredit         rs_GoodCredit   
 1         rs_GoodCredit  LoanOverThreshold  rs_LoanOverThreshold   
 2  rs_LoanOverThreshold        SmallAmount        rs_SmallAmount   
 3        rs_SmallAmount        IsNewCredit        rs_IsNewCredit   
 4    initial_source_log    StartWithCreate    rs_StartWithCreate   
 5    initial_source_log   ModerateDuration   rs_ModerateDuration   
 
                 labels  
 0         [GoodCredit]  
 1  [LoanOverThreshold]  
 2        [SmallAmount]  
 3        [IsNewCredit]  
 4    [StartWithCreate]  
 5   [ModerateDuration]  ,
 'queries':                query                              predicates
 0         GoodCredit                    (CreditScore >= 600)
 1  LoanOverThreshold              (RequestedAmount >= 10000)
 2        SmallAmount               (RequestedAmount < 15000)
 3        IsNewCredit   (ApplicationType in { 'New credit'

In [4]:
import dash
from dash import html, dcc, Input, Output, State, ctx, dash_table
import pandas as pd
import threading
import flask

# Load and prepare registry
registry_raw = log_view.query_registry.summary()
registry = registry_raw['evaluations'].copy()
registry['labels'] = registry['labels'].apply(lambda x: ', '.join(x) if isinstance(x, list) else str(x))

# Global variables
selected_filters = []
selected_sequence_df = pd.DataFrame()
shutdown_pending = False

app = dash.Dash(__name__)
server = app.server

app.layout = html.Div([
    html.H2("Step-by-Step Filter Builder"),

    html.Div(id='instruction', children="Start by selecting a filter from the original log."),

    dash_table.DataTable(
        id='registry-table',
        columns=[{"name": i, "id": i} for i in registry.columns],
        data=[], 
        row_selectable='single',
        selected_rows=[],
        page_size=10,
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left'}
    ),

    html.Button("Add Filter to Chain", id="confirm-button", n_clicks=0),

    html.Hr(),

    html.Div(id='selected-output')
])


def shutdown_server():
    func = flask.request.environ.get("werkzeug.server.shutdown")
    if func:
        func()
    else:
        print("Shutdown function not found.")


@app.callback(
    Output('registry-table', 'data'),
    Output('registry-table', 'selected_rows'),
    Output('instruction', 'children'),
    Output('selected-output', 'children'),
    Input('confirm-button', 'n_clicks'),
    State('registry-table', 'derived_virtual_selected_rows'),
    State('registry-table', 'data')
)
def update_table(n_clicks, selected_rows, table_data):
    global selected_filters, shutdown_pending, selected_sequence_df

    if shutdown_pending:
        return [], [], "Sequence complete. Shutting down...", dash_table.DataTable()

    if n_clicks == 0 or not selected_rows:
        initial = registry[registry['source_log'] == 'initial_source_log']
        return initial.to_dict('records'), [], "Start by selecting a filter from the original log.", "No filters selected yet."

    selected_row = table_data[selected_rows[0]]
    selected_filters.append(selected_row)
    selected_sequence_df = pd.DataFrame(selected_filters)

    # Determine next options
    last_result_set = selected_row['result_set']
    next_filters = registry[registry['source_log'] == last_result_set]

    # Display current sequence
    sequence_table = dash_table.DataTable(
        columns=[{"name": i, "id": i} for i in selected_sequence_df.columns],
        data=selected_sequence_df.to_dict('records'),
        style_cell={'textAlign': 'left'},
        style_table={'overflowX': 'auto'}
    )

    if next_filters.empty:
        print("\n Final filter chain:")
        print(selected_sequence_df)

        shutdown_pending = True
        threading.Timer(1.0, lambda: flask.request.environ.get('werkzeug.server.shutdown')()).start()

        return [], [], "No more filters. Closing app...", html.Div([
            html.H4("Final Filter Sequence:"),
            sequence_table
        ])

    return next_filters.to_dict('records'), [], f"Select the next filter from: {last_result_set}", html.Div([
        html.H4("Current Filter Sequence:"),
        sequence_table
    ])


if __name__ == '__main__':
    app.run(debug=True)



 Final filter chain:
             source_log              query            result_set  \
0    initial_source_log         GoodCredit         rs_GoodCredit   
1         rs_GoodCredit  LoanOverThreshold  rs_LoanOverThreshold   
2  rs_LoanOverThreshold        SmallAmount        rs_SmallAmount   
3        rs_SmallAmount        IsNewCredit        rs_IsNewCredit   

              labels  
0         GoodCredit  
1  LoanOverThreshold  
2        SmallAmount  
3        IsNewCredit  


In [5]:
selected_sequence_df

,source_log,query,result_set,labels
0,initial_source_log,GoodCredit,rs_GoodCredit,GoodCredit
1,rs_GoodCredit,LoanOverThreshold,rs_LoanOverThreshold,LoanOverThreshold
2,rs_LoanOverThreshold,SmallAmount,rs_SmallAmount,SmallAmount
3,rs_SmallAmount,IsNewCredit,rs_IsNewCredit,IsNewCredit


In [6]:
from logview.predicate import Query

def recursively_apply_filters(selected_sequence_df, log_view):
    """
    Apply each filter to all current subsets (starting with the initial log),
    splitting each subset into result and complement at every step.

    Returns:
        A DataFrame with:
            - subset_name
            - label_path
            - num_cases
            - avg_case_duration_seconds
    """
    results = []

    def case_stats(df, name, label_path):
        if df.empty:
            return {
                "subset_name": name,
                "label_path": " → ".join(label_path),
                "num_cases": 0,
                "avg_case_duration_seconds": 0
            }

        durations = (
            df.groupby("case:concept:name")["time:timestamp"]
            .agg(["min", "max"])
            .apply(lambda row: (row["max"] - row["min"]).total_seconds(), axis=1)
        )

        return {
            "subset_name": name,
            "label_path": " → ".join(label_path),
            "num_cases": df["case:concept:name"].nunique(),
            "avg_case_duration_seconds": durations.mean()
        }

    # Start with the initial log
    initial_log_name = selected_sequence_df.iloc[0]['source_log']
    initial_df = log_view.result_set_name_cache[initial_log_name]
    current_subsets = [{
        "df": initial_df,
        "name": initial_log_name,
        "label_path": ["Initial Source"]
    }]

    # Apply each filter recursively
    for i, row in selected_sequence_df.iterrows():
        next_subsets = []
        query_name = row["query"]
        filter_label = row["labels"]

        # Retrieve the original Query object based on its name
        query_obj = None
        for result_set_id in log_view.query_registry.get_registered_result_set_ids():
            evaluation = log_view.query_registry.get_evaluation(result_set_id)
            if evaluation["query"].name == query_name:  # Fixed attribute access
                query_obj = evaluation["query"]
                break

        if query_obj is None:
            raise ValueError(f"Query '{query_name}' not found in registry.")

        for subset in current_subsets:
            subset_df = subset["df"]
            subset_name = subset["name"]
            path = subset["label_path"]

            # Apply the query directly on this subset
            df_filtered, df_complement = log_view.query_evaluator.evaluate(subset_df, query_obj)

            # Create names and paths
            filtered_name = f"{subset_name}_F{i+1}"
            complement_name = f"{subset_name}_C{i+1}"
            path_filtered = path + [f"{filter_label} ✓"]
            path_complement = path + [f"{filter_label} ✗"]

            # Store stats
            results.append(case_stats(df_filtered, filtered_name, path_filtered))
            results.append(case_stats(df_complement, complement_name, path_complement))

            # Queue up for the next round
            next_subsets.append({
                "df": df_filtered,
                "name": filtered_name,
                "label_path": path_filtered
            })
            next_subsets.append({
                "df": df_complement,
                "name": complement_name,
                "label_path": path_complement
            })

        current_subsets = next_subsets

    return pd.DataFrame(results)


In [7]:
tree_stats_df = recursively_apply_filters(selected_sequence_df, log_view)
display(tree_stats_df)

,subset_name,label_path,num_cases,avg_case_duration_seconds
0,initial_source_log_F1,Initial Source → GoodCredit ✓,15238,1.610002e+06
1,initial_source_log_C1,Initial Source → GoodCredit ✗,16271,2.156251e+06
2,initial_source_log_F1_F2,Initial Source → GoodCredit ✓ → LoanOverThresh...,10084,1.687141e+06
3,initial_source_log_F1_C2,Initial Source → GoodCredit ✓ → LoanOverThresh...,5154,1.459078e+06
4,initial_source_log_C1_F2,Initial Source → GoodCredit ✗ → LoanOverThresh...,10020,2.169071e+06
5,initial_source_log_C1_C2,Initial Source → GoodCredit ✗ → LoanOverThresh...,6251,2.135700e+06
6,initial_source_log_F1_F2_F3,Initial Source → GoodCredit ✓ → LoanOverThresh...,2529,1.538752e+06
7,initial_source_log_F1_F2_C3,Initial Source → GoodCredit ✓ → LoanOverThresh...,7555,1.736813e+06
8,initial_source_log_F1_C2_F3,Initial Source → GoodCredit ✓ → LoanOverThresh...,5154,1.459078e+06
9,initial_source_log_F1_C2_C3,Initial Source → GoodCredit ✓ → LoanOverThresh...,0,0.000000e+00


In [8]:
def prepare_sunburst_data_for_sunburst(tree_stats_df):
    """
    Prepare only leaf nodes from the recursive tree to be shown in plotly.sunburst().
    Returns a DataFrame with Level1, Level2, ..., and metrics.
    """
    # Split label path
    tree_stats_df['label_list'] = tree_stats_df['label_path'].str.split(" → ")

    # Identify non-leaf nodes (those that are prefixes of other paths)
    all_paths = tree_stats_df['label_list'].tolist()
    all_path_strings = set(" → ".join(path) for path in all_paths)

    is_leaf = []
    for path in all_paths:
        path_str = " → ".join(path)
        has_child = any(
            other != path_str and other.startswith(path_str + " → ")
            for other in all_path_strings
        )
        is_leaf.append(not has_child)

    # Keep only leaf nodes
    leaf_df = tree_stats_df[is_leaf].copy()

    # Split again for sunburst levels
    level_df = leaf_df['label_path'].str.split(" → ", expand=True)
    level_df.columns = [f'Level{i+1}' for i in range(level_df.shape[1])]

    sunburst_df = pd.concat([level_df, leaf_df[['num_cases', 'avg_case_duration_seconds']]], axis=1)
    return sunburst_df


In [9]:
sunburst_df = prepare_sunburst_data_for_sunburst(tree_stats_df)
display(sunburst_df)

,Level1,Level2,Level3,Level4,Level5,num_cases,avg_case_duration_seconds
14,Initial Source,GoodCredit ✓,LoanOverThreshold ✓,SmallAmount ✓,IsNewCredit ✓,2461,1.548907e+06
15,Initial Source,GoodCredit ✓,LoanOverThreshold ✓,SmallAmount ✓,IsNewCredit ✗,68,1.171228e+06
16,Initial Source,GoodCredit ✓,LoanOverThreshold ✓,SmallAmount ✗,IsNewCredit ✓,7288,1.754820e+06
17,Initial Source,GoodCredit ✓,LoanOverThreshold ✓,SmallAmount ✗,IsNewCredit ✗,267,1.245295e+06
18,Initial Source,GoodCredit ✓,LoanOverThreshold ✗,SmallAmount ✓,IsNewCredit ✓,4976,1.472006e+06
19,Initial Source,GoodCredit ✓,LoanOverThreshold ✗,SmallAmount ✓,IsNewCredit ✗,178,1.097686e+06
20,Initial Source,GoodCredit ✓,LoanOverThreshold ✗,SmallAmount ✗,IsNewCredit ✓,0,0.000000e+00
21,Initial Source,GoodCredit ✓,LoanOverThreshold ✗,SmallAmount ✗,IsNewCredit ✗,0,0.000000e+00
22,Initial Source,GoodCredit ✗,LoanOverThreshold ✓,SmallAmount ✓,IsNewCredit ✓,2158,2.294083e+06
23,Initial Source,GoodCredit ✗,LoanOverThreshold ✓,SmallAmount ✓,IsNewCredit ✗,351,1.490905e+06


In [10]:
import plotly.express as px

fig = px.sunburst(
    sunburst_df,
    path=[col for col in sunburst_df.columns if col.startswith("Level")],
    values='num_cases',
    color='avg_case_duration_seconds',
    color_continuous_scale='Blues',
    title='Case Breakdown by Filter Path'
)
fig.update_layout(margin=dict(t=40, l=0, r=0, b=0))
fig.show()